# {{ALGORITHM_NAME}} with OpenEO

This notebook demonstrates {{ALGORITHM_DESCRIPTION}} using Sentinel-2 imagery with the OpenEO API.

## Overview

In this notebook, we will:
1. Load parameters using the OpenEO UDP parameter management system
2. Connect to an OpenEO backend service with automatic endpoint selection
3. Define an area of interest and temporal range
4. Load Sentinel-2 L2A imagery with validated parameters
5. Implement the {{ALGORITHM_NAME}} algorithm
6. Create visualizations and export results

## Algorithm Information

{{ALGORITHM_DETAILED_DESCRIPTION}}

### Applications

{{ALGORITHM_APPLICATIONS}}

## Import Required Libraries

Import the necessary Python libraries for data processing, visualization, and parameter management.

In [ ]:
# Standard OpenEO and visualization imports
import openeo
import matplotlib.pyplot as plt
from PIL import Image
from openeo.processes import array_create, if_, and_
from openeo.api.process import Parameter

# OpenEO UDP parameter management system
from openeo_udp import ParameterManager
from openeo_udp.config import get_connection, load_endpoint_config

## Load Parameters and Configuration

Load algorithm-specific parameter sets from the co-located .py file and set up endpoint configuration.

In [ ]:
# Initialize parameter manager for this notebook
import os
current_notebook = os.path.basename(__file__) if '__file__' in globals() else 'current_notebook.ipynb'
param_manager = ParameterManager(current_notebook)

# Show available parameter sets
available_sets = param_manager.list_parameter_sets()
print(f"Available parameter sets: {available_sets}")

# Use the first parameter set (or specify one)
if available_sets:
    param_manager.use_parameter_set(available_sets[0])
    current_params = param_manager.get_parameter_set()
    print(f"Using parameter set: {available_sets[0]}")
    print(f"Parameters: {current_params}")
else:
    print("No parameter sets found")

# Load endpoint configuration
from openeo_udp.config import get_connection, load_endpoint_config
endpoint_config = load_endpoint_config()
available_endpoints = [name for name, config in endpoint_config['endpoints'].items() 
                      if config.get('enabled', True)]
print(f"Available endpoints: {available_endpoints}")

## Select Endpoint and Connect

Automatically select a compatible endpoint and establish connection with proper authentication.

In [ ]:
# Find compatible endpoint
selected_endpoint = available_endpoints[0] if available_endpoints else "eopf_explorer"
print(f"Selected endpoint: {selected_endpoint}")

# Establish connection
try:
    connection = get_connection(selected_endpoint)
    print(f"Successfully connected to {selected_endpoint}")
except Exception as e:
    print(f"Connection failed: {e}")
    print("Falling back to first available endpoint...")
    connection = get_connection()
    
# Set the endpoint for parameter formatting
param_manager.set_endpoint(selected_endpoint)

## Load Algorithm Parameters

Load and validate algorithm-specific parameters with endpoint-compatible formatting.

In [ ]:
# Load standard parameters (automatically formatted for selected endpoint)
bounding_box = param_manager.get_parameter('bounding_box')
time = param_manager.get_parameter('time')
bands = param_manager.get_parameter('bands')

# Display parameter values
print("Parameter Configuration:")
print(f"Spatial extent: {bounding_box.default}")
print(f"Temporal range: {time.default}")
print(f"Bands: {bands.default}")

# Load optional parameters if available
try:
    cloud_cover_threshold = param_manager.get_parameter('cloud_cover_threshold')
    print(f"Cloud cover threshold: {cloud_cover_threshold.default}%")
except ValueError:
    # Use default if parameter not found
    cloud_cover_threshold = Parameter("cloud_cover_threshold", default=30, optional=True)
    print("Using default cloud cover threshold: 30%")

# Show alternative test areas if available
alternative_areas = param_manager.get_alternative_areas()
if alternative_areas:
    print(f"\nAlternative test areas available: {list(alternative_areas.keys())}")
    print("To use an alternative area, modify the bounding_box parameter above.")

## Load Sentinel-2 Data

Load Sentinel-2 L2A (atmospherically corrected) data with validated parameters and endpoint-specific formatting.

In [ ]:
# Get endpoint-specific collection information
endpoint_info = endpoint_config['endpoints'][selected_endpoint]
collection_id = endpoint_info['collection_id']

print(f"Loading collection: {collection_id}")
print(f"Using bands: {bands.default}")

# Load data with cloud filtering
s2cube = connection.load_collection(
    collection_id,
    temporal_extent=time.default,
    spatial_extent=bounding_box.default,
    bands=bands.default,
    properties={
        "eo:cloud_cover": lambda x: x < cloud_cover_threshold.default,
    },
)

# Apply pixel selection if supported by endpoint
if 'apply_pixel_selection' in endpoint_info.get('capabilities', []):
    s2cube = s2cube.process(
        "apply_pixel_selection",
        pixel_selection="first",
        data=s2cube,
    )

print("Sentinel-2 data loaded successfully")

## Implement {{ALGORITHM_NAME}} Algorithm

{{ALGORITHM_IMPLEMENTATION_DESCRIPTION}}

*Note: Replace this section with your specific algorithm implementation.*

In [ ]:
def algorithm_function(data):
    """
    Implement your algorithm here.
    
    Args:
        data: Input data array from Sentinel-2 bands
        
    Returns:
        Processed result for visualization
    """
    # TODO: Replace with actual algorithm implementation
    # This is a placeholder that extracts the first three bands for RGB visualization
    
    # Extract bands (adapt indices based on your algorithm's needs)
    b1, b2, b3 = data[0], data[1], data[2]
    
    # Example processing: create true color composite
    true_color = array_create([b3 * 3, b2 * 3, b1 * 3])  # RGB composite
    
    return true_color

# Apply algorithm to the data
result_image = s2cube.apply_dimension(dimension="bands", process=algorithm_function)

print("Algorithm applied successfully")

## Visualization and Export

Process the results for visualization and create output products.

In [ ]:
# Get visualization parameters
try:
    viz_config = param_manager.get_parameter('visualization_config')
    viz_params = viz_config.default
except ValueError:
    # Use default visualization parameters
    viz_params = {"input_min": 0, "input_max": 0.8, "output_min": 0, "output_max": 255, "format": "PNG"}

# Scale for visualization
result_scaled = result_image.linear_scale_range(
    input_min=viz_params["input_min"],
    input_max=viz_params["input_max"], 
    output_min=viz_params["output_min"],
    output_max=viz_params["output_max"]
)

# Save as PNG
result_png = result_scaled.save_result(viz_params.get("format", "PNG"))

print("Results prepared for visualization")

## Create XYZ Tile Service (Optional)

Create a web mapping service for interactive visualization if supported by the endpoint.

In [ ]:
# Check if endpoint supports service creation
if 'create_service' in endpoint_info.get('capabilities', []):
    try:
        # Create XYZ service for web mapping
        service = connection.create_service(
            {
                "process_graph": result_png.flat_graph(),
                "parameters": [
                    time.to_dict(),
                    bounding_box.to_dict(),
                    bands.to_dict(),
                ]
            },
            id=f"{metadata.get('algorithm_id', 'algorithm')}_service",
            type="XYZ",
            title=f"OpenEO UDP - {metadata.get('algorithm_name', 'Algorithm')}",
            description=f"{metadata.get('description', 'Algorithm visualization')}",
            configuration={
                "tile_size": 256,
                "minzoom": 5,
                "maxzoom": 14,
                "extent": [
                    bounding_box.default["west"],
                    bounding_box.default["south"],
                    bounding_box.default["east"],
                    bounding_box.default["north"],
                ],
                "scope": "public",
            },
        )
        
        print(f"XYZ Service created: {service.service_id}")
        service_id = service.service_id
        
    except Exception as e:
        print(f"Service creation failed: {e}")
        service_id = None
else:
    print("Service creation not supported by this endpoint")
    service_id = None

## Download and Display Results

Download a sample of the results for local visualization.

In [ ]:
# Define output filename
algorithm_id = metadata.get('algorithm_id', 'result')
output_filename = f"{algorithm_id}_result.png"

# Download result
try:
    connection.download(
        {
            "process_graph": result_png.flat_graph(),
            "parameters": [
                time.to_dict(),
                bounding_box.to_dict(),
                bands.to_dict(),
            ]
        },
        output_filename,
    )
    
    # Display the result
    result_img = Image.open(output_filename)
    plt.figure(figsize=(12, 10))
    plt.imshow(result_img)
    plt.title(f"{metadata.get('algorithm_name', 'Algorithm')} Results", fontsize=16)
    plt.axis("off")
    plt.show()
    
    print(f"Results saved to: {output_filename}")
    
except Exception as e:
    print(f"Download failed: {e}")

## Alternative Test Areas

Test the algorithm with different geographical areas using the predefined alternative locations.

In [ ]:
# Display available alternative test areas
alternative_areas = param_manager.get_alternative_areas()

if alternative_areas:
    print("Available alternative test areas:")
    print("=" * 40)
    
    for area_name, area_config in alternative_areas.items():
        spatial_extent = area_config.get('spatial_extent', area_config)
        description = area_config.get('description', 'No description available')
        recommended_dates = area_config.get('recommended_dates', [])
        notes = area_config.get('notes', '')
        
        print(f"\n{area_name}:")
        print(f"  Description: {description}")
        print(f"  Spatial extent: {spatial_extent}")
        if recommended_dates:
            print(f"  Recommended dates: {recommended_dates}")
        if notes:
            print(f"  Notes: {notes}")
    
    print("\n" + "=" * 40)
    print("To test with an alternative area:")
    print("1. Copy the spatial extent from above")
    print("2. Update the bounding_box parameter")
    print("3. Optionally update the time parameter")
    print("4. Re-run the data loading and processing cells")

else:
    print("No alternative test areas defined for this algorithm.")

## Export Process Graph

Export the algorithm as a reusable OpenEO User-Defined Process.

In [ ]:
# Create process graph for reuse
import json

process_graph = {
    "process_graph": result_png.flat_graph(),
    "parameters": [
        time.to_dict(),
        bounding_box.to_dict(),
        bands.to_dict(),
    ],
    "id": metadata.get('algorithm_id', 'algorithm'),
    "summary": metadata.get('algorithm_name', 'Algorithm'),
    "description": metadata.get('description', 'OpenEO UDP algorithm'),
    "categories": [metadata.get('category', 'other')],
    "experimental": True,
    "exceptions": {},
    "examples": [],
    "links": [],
}

# Save process graph
process_graph_filename = f"{algorithm_id}_process_graph.json"
with open(process_graph_filename, "w") as f:
    json.dump(process_graph, f, indent=2)

print(f"Process graph exported to: {process_graph_filename}")
print(f"Process ID: {process_graph['id']}")

# Display compatibility information
compatibility = param_manager.check_endpoint_compatibility(selected_endpoint)
print(f"\nEndpoint compatibility for {selected_endpoint}:")
print(f"- Supported: {'Yes' if compatibility['is_supported'] else 'No'}")
if compatibility['issues']:
    print(f"- Issues: {', '.join(compatibility['issues'])}")
if 'formatted_bands' in compatibility:
    print(f"- Formatted bands: {compatibility['formatted_bands']}")

## Summary and Next Steps

This notebook successfully demonstrated the {{ALGORITHM_NAME}} algorithm using the OpenEO UDP parameter management system.

### Key Features Used:
- ✅ Automatic parameter discovery and validation
- ✅ Endpoint compatibility checking and connection management  
- ✅ Parameter formatting for different OpenEO backends
- ✅ Alternative test area support
- ✅ Process graph export for reusability

### Next Steps:
1. **Algorithm Development**: Replace the placeholder algorithm implementation with your specific processing logic
2. **Parameter Tuning**: Adjust parameters in the .params.yaml file for optimal results
3. **Validation**: Test with different endpoints and alternative areas
4. **Documentation**: Add detailed algorithm description and usage examples
5. **Integration**: Register the process graph as a User-Defined Process

### Parameter Management Benefits:
- **Consistency**: Standardized parameter handling across notebooks
- **Flexibility**: Easy switching between endpoints and test areas  
- **Validation**: Automatic parameter validation and error checking
- **Reusability**: Process graphs with embedded parameters for sharing
- **Maintainability**: Centralized configuration management

For more information about the parameter management system, see the OpenEO UDP documentation.